In [ ]:
pip install opencv-python

In [ ]:
pip install matplotlib

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

import math

## Фильтр для видео

In [2]:
def func(img, numb):
    #get image with blur
    return cv.GaussianBlur(img, (numb, numb), 1.5)

def cut_img(img):
    #cut image in midle line
    img = np.concatenate((img[:, 320:], img[:, :320]), axis=1)
    return img
    
def waves(img):
    rows, cols, numb = img.shape
    img_output = np.zeros(img.shape, dtype=img.dtype)
    for i in range(rows):
        for j in range(cols):
            offset_x = int(20.0 * math.sin(2 * 3.14 * i / 150))
            offset_y = int(20.0 * math.cos(2 * 3.14 * j / 150))
            if i+offset_y < rows and j+offset_x < cols:
                img_output[i,j] = img[(i+offset_y)%rows,(j+offset_x)%cols]
            else:
                img_output[i,j] = 0
    return img_output

def noise(img):
    N =140*180
    x = np.random.random_integers(480, size=N)-1
    y = np.random.random_integers(640, size=N)-1
    for i in range(len(x)//2):
        img[ x[i], y[i] ] = [255,255,255] # black
        img[ x[N-i-1], y[N-i-1] ] = [0,0,0] # white
    return img

In [6]:
# create window
cv.namedWindow( "video filter" )

# 0 - open default camera
cap = cv.VideoCapture(0)
filter = cv.COLOR_BGR2GRAY

numb = 1 #level of blur
numb_f2 = 1 #for cut and waves

font = cv.FONT_HERSHEY_SIMPLEX
fontScale = 0.8
lineType = cv.LINE_4

#d - change parts of image

while True:
    
    #for waves
    if numb_f2 == 3:
        flag, img = cap.read()
        img = waves(img)
    #for cut img
    elif numb_f2 == 4:
        flag, img = cap.read()
        img = cut_img(img)
    #get noise
    elif numb_f2 == 5:
        flag, img = cap.read()
        img = noise(img)
    else:
        flag, img = cap.read()

    #text about filters
    cv.putText(img,'1 - no blur', (20, 30), font, 
                fontScale,(0,0,225),lineType);
    cv.putText(img,'3 - small blur', (20, 60), font, 
                fontScale,(0,0,255),lineType);
    cv.putText(img,'7 - big blur', (20, 90), font, 
                fontScale, (0,0,255),lineType);
    cv.putText(img,'s - waves=)', (20, 120), font, 
                fontScale,(0,0,255),lineType);
    cv.putText(img,'d - cut img', (20, 150), font, 
                fontScale, (0,0,255),lineType); 
    cv.putText(img,'z - no effects', (20, 180), font,
                fontScale,(0,0,255),lineType);  
    cv.putText(img,'f - noise', (20, 210), font,
                fontScale,(0,0,255),lineType);         

    #for filterd BGR2BGRA and BGR2GRAY
    img_with_filter = cv.cvtColor(img, cv.COLOR_BGR2BGRA)
    #for Canny and blur
    new_img = func(img_with_filter, numb) #1 - размытие

    try:
        cv.imshow("video filter", new_img)
    except:
        cap.release()
        raise
    
    k = cv.waitKey(10)
    #waavves
    if k == 115: #s
        numb_f2 = 3

    #change parts of image (cut img)
    if k == 100: #d
        numb_f2 = 4

    if k == 102: #f
        numb_f2 = 5

    #clear img (without cut, waves, Canny)
    if k == 122:#z
        numb_f2 = 1

    #level of blur
    elif k == 49: #1
        numb = 1

    elif k == 51: #3
        numb = 3

    elif k == 55: #7
        numb = 7

    #exit
    if k == 113:
        break

cap.release()
cv.destroyAllWindows()